In [5]:
import os 
import pandas as pd 
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.over_sampling import BorderlineSMOTE
import dask.dataframe as dd
import fastparquet
from sklearn.svm import OneClassSVM
try:
    os.chdir("C:/Users/Sam/Documents/SISE/Fouille de données")
except:
    os.chdir("/Users/titouanhoude/Documents/GitHub")
    
train = pd.read_parquet('train.parquet.gzip')
test = pd.read_parquet('test.parquet.gzip')
train_test = train.sample(n = 100000)
test_test = test.sample(n = 100000)


In [3]:
train_test["FlagImpaye"].value_counts()

0    99382
1      618
Name: FlagImpaye, dtype: int64

In [15]:
Xtrain = train_test.drop(["FlagImpaye", "ZIBZIN", "Date", "Heure_split", "DateTransaction", "Unnamed: 0"], axis = 1)

Ytrain = pd.DataFrame(train_test.FlagImpaye)
Ytrain = train_test['FlagImpaye'].astype('int')

Xtest  = test_test.drop(["FlagImpaye","ZIBZIN", "Date", "Heure_split", "DateTransaction", 'Unnamed: 0'], axis = 1)
Ytest  = test_test.FlagImpaye

In [18]:
from sklearn.utils import class_weight


In [25]:
classes = np.unique(Ytrain)
cw = class_weight.compute_class_weight(class_weight = 'balanced',classes =  np.unique(Ytrain),y= Ytrain)
weights = dict(zip(classes,cw))

In [34]:
from xgboost import XGBRFClassifier
model = XGBRFClassifier(class_weight=weights)
model.fit(Xtrain, Ytrain)
y_score = model.predict_proba(Xtest)
y_pred = model.predict(Xtest)

[00:53:33] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "class_weight" } are not used.



In [35]:
from sklearn.metrics import classification_report
tab = classification_report(Ytest, y_pred)
print(tab)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     99072
           1       0.98      0.66      0.79       928

    accuracy                           1.00    100000
   macro avg       0.99      0.83      0.89    100000
weighted avg       1.00      1.00      1.00    100000



In [51]:
def sampling_strategy(X,y,n_samples, t='majority'):
    target_classes = ''
    if t == 'majority':
        target_classes = y.value_counts() > n_samples
    elif t == 'minority':
        target_classes = y.value_counts() < n_samples
    tc = target_classes[target_classes == True].index
    #target_classes_all = y.value_counts().index
    sampling_strategy = {}
    for target in tc:
        sampling_strategy[target] = n_samples
    return sampling_strategy

count = train_test.FlagImpaye.value_counts()
print(count)
n_samples = int(count.median())


0    99391
1      609
Name: FlagImpaye, dtype: int64


In [53]:
from imblearn.under_sampling import OneSidedSelection
under_sampler = OneSidedSelection(sampling_strategy=sampling_strategy(Xtrain, Ytrain,n_samples,t='majority'))
X_under, y_under = under_sampler.fit_resample(Xtrain, Ytrain)

ValueError: The 'sampling_strategy' parameter of OneSidedSelection must be a float in the range (0, 1], a str among {'auto', 'majority', 'not majority', 'not minority', 'all'}, an instance of 'list' or a callable. Got {0: 50000} instead.

In [60]:
from imblearn.over_sampling import SMOTE
over_sampler = SMOTE(sampling_strategy=sampling_strategy(Xtrain, Ytrain,n_samples, t='minority'),k_neighbors=3)
X_bal, y_bal = over_sampler.fit_resample(Xtrain, Ytrain)

In [57]:
from xgboost import XGBRFClassifier
from sklearn.ensemble import GradientBoostingClassifier
model = XGBRFClassifier(class_weight=weights)
model.fit(X_bal, y_bal )
y_score = model.predict_proba(Xtest)
y_pred = model.predict(Xtest)

[01:24:53] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "class_weight" } are not used.



In [58]:
from sklearn.metrics import classification_report
tab = classification_report(Ytest, y_pred)
print(tab)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     99072
           1       0.57      0.69      0.62       928

    accuracy                           0.99    100000
   macro avg       0.78      0.84      0.81    100000
weighted avg       0.99      0.99      0.99    100000



In [64]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier()
model.fit(Xtrain, Ytrain)
y_score = model.predict_proba(Xtest)
y_pred = model.predict(Xtest)

In [65]:
from sklearn.metrics import classification_report
tab = classification_report(Ytest, y_pred)
print(tab)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     99072
           1       0.90      0.65      0.76       928

    accuracy                           1.00    100000
   macro avg       0.95      0.83      0.88    100000
weighted avg       1.00      1.00      1.00    100000



In [67]:
train_test = train.sample(n = 100000, weights=train['FlagImpaye'].map(weights))
test_test = test.sample(n = 100000,weights=test['FlagImpaye'].map(weights))

Xtrain = train_test.drop(["FlagImpaye", "ZIBZIN", "Date", "Heure_split", "DateTransaction", "Unnamed: 0"], axis = 1)

Ytrain = pd.DataFrame(train_test.FlagImpaye)
Ytrain = train_test['FlagImpaye'].astype('int')

Xtest  = test_test.drop(["FlagImpaye","ZIBZIN", "Date", "Heure_split", "DateTransaction", 'Unnamed: 0'], axis = 1)
Ytest  = test_test.FlagImpaye

In [111]:
from xgboost import XGBRFClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
model = RandomForestClassifier(
                        class_weight =weights)
model.fit(Xtrain, Ytrain)
y_score = model.predict_proba(Xtest)
y_pred = model.predict(Xtest)
from sklearn.metrics import classification_report
tab = classification_report(Ytest, y_pred)
print(tab)

              precision    recall  f1-score   support

           0       0.98      1.00      0.99     93427
           1       0.98      0.69      0.81      6573

    accuracy                           0.98    100000
   macro avg       0.98      0.85      0.90    100000
weighted avg       0.98      0.98      0.98    100000



In [114]:
from sklearn.ensemble import BaggingClassifier
bagging_clf = BaggingClassifier(base_estimator=model,
                                random_state=0)
bagging_clf.fit(Xtrain, Ytrain)
y_score = bagging_clf.predict_proba(Xtest)
y_pred = bagging_clf.predict(Xtest)
from sklearn.metrics import classification_report
tab = classification_report(Ytest, y_pred)
print(tab)

              precision    recall  f1-score   support

           0       0.98      1.00      0.99     93427
           1       0.99      0.69      0.81      6573

    accuracy                           0.98    100000
   macro avg       0.98      0.84      0.90    100000
weighted avg       0.98      0.98      0.98    100000



In [115]:
model = XGBRFClassifier()

from sklearn.ensemble import BaggingClassifier
bagging_clf = BaggingClassifier(base_estimator=model,
                                random_state=0)
bagging_clf.fit(Xtrain, Ytrain)
y_score = bagging_clf.predict_proba(Xtest)
y_pred = bagging_clf.predict(Xtest)
from sklearn.metrics import classification_report
tab = classification_report(Ytest, y_pred)
print(tab)

              precision    recall  f1-score   support

           0       0.98      1.00      0.99     93427
           1       0.95      0.70      0.81      6573

    accuracy                           0.98    100000
   macro avg       0.97      0.85      0.90    100000
weighted avg       0.98      0.98      0.98    100000



In [ ]:
from sklearn.svm import SVC

model = SVC(class_weight =weights)
model.fit(Xtrain, Ytrain)
y_score = model.predict_proba(Xtest)
y_pred = model.predict(Xtest)
from sklearn.metrics import classification_report
tab = classification_report(Ytest, y_pred)
print(tab)

In [100]:
from sklearn.model_selection import StratifiedKFold
classifier = XGBRFClassifier(max_depth=6,gamma = 1, learning_rate =  0.3)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [101]:
from sklearn.model_selection import cross_validate
cv_results_ = cross_validate(classifier, Xtrain, Ytrain, cv=cv,
                                                     scoring=['f1'],
                                                     return_estimator=True)

In [98]:
classifier_0 = cv_results_['estimator'][0]
classifier_0

XGBRFClassifier(base_score=None, booster=None, callbacks=None,
                colsample_bylevel=None, colsample_bytree=None,
                early_stopping_rounds=None, enable_categorical=False,
                eval_metric=None, feature_types=None, gamma=None, gpu_id=None,
                grow_policy=None, importance_type=None,
                interaction_constraints=None, learning_rate=0.01, max_bin=None,
                max_cat_threshold=None, max_cat_to_onehot=None,
                max_delta_step=None, max_depth=2, max_leaves=None,
                min_child_weight=None, missing=nan, monotone_constraints=None,
                n_estimators=132, n_jobs=-1, num_parallel_tree=None,
                objective='binary:logistic', predictor=None, random_state=0, ...)

In [99]:
results = round(pd.DataFrame(cv_results_),3)
results

,fit_time,score_time,estimator,test_f1
0,1.224,0.012,"XGBRFClassifier(base_score=None, booster=None,...",0.789
1,1.229,0.012,"XGBRFClassifier(base_score=None, booster=None,...",0.789
2,1.232,0.012,"XGBRFClassifier(base_score=None, booster=None,...",0.796
3,1.253,0.015,"XGBRFClassifier(base_score=None, booster=None,...",0.795
4,1.238,0.012,"XGBRFClassifier(base_score=None, booster=None,...",0.791


In [89]:
from sklearn.model_selection import cross_validate
cv_results_ = cross_validate(classifier_0, Xtrain, Ytrain, cv=cv,
                                                     scoring=['roc_auc',
                                                              'average_precision',
                                                              'balanced_accuracy'],
                                                     strategy_name="Cost-sensitive")

TypeError: cross_validate() got an unexpected keyword argument 'strategy_name'